# Shortwave Radiation Transfer Example

This notebook demonstrates how to compute shortwave radiation fluxes using the `pyrte_rrtmgp` library. It downloads the required data, computes the gas optics using a two-stream approximation, solves the radiation transfer equations, and validates the results against reference datasets.

In [12]:
import os
import multiprocessing

import numpy as np
import xarray as xr
import dask

## Importing pyrte_rrtmgp Modules

We import the necessary modules from the `pyrte_rrtmgp` package which provide tools for gas optics and solving the radiation transfer equations.

In [3]:
from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver

ERROR_TOLERANCE = 1e-7

## Data Download and Directory Setup

Download the RRTMGP data and define paths for the input and reference directories.

In [13]:
rte_rrtmgp_dir = download_rrtmgp_data()
rfmip_dir = os.path.join(rte_rrtmgp_dir, "examples", "rfmip-clear-sky")
input_dir = os.path.join(rfmip_dir, "inputs")
ref_dir = os.path.join(rfmip_dir, "reference")
os.listdir(rte_rrtmgp_dir)

['rrtmgp-gas-lw-g256.nc',
 'LICENSE',
 'rrtmgp-clouds-lw-bnd.nc',
 'CITATION.cff',
 'rrtmgp-clouds-lw-g256.nc',
 'rrtmgp-aerosols-merra-lw.nc',
 'rrtmgp-clouds-sw-bnd.nc',
 'README.md',
 'rrtmgp-clouds-sw-g224.nc',
 'rrtmgp-gas-lw-g128.nc',
 '.gitignore',
 'rrtmgp-aerosols-merra-sw.nc',
 'examples',
 'rrtmgp-clouds-sw-g112.nc',
 'rrtmgp-clouds-lw-g128.nc',
 'rrtmgp-gas-sw-g224.nc',
 'rrtmgp-gas-sw-g112.nc']

## Loading Gas Optics Data and Atmosphere Dataset

Load the shortwave gas optics file and the associated atmosphere dataset that will be used for the simulation.

In [14]:
gas_optics_sw = load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)

atmosphere_file = "multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
atmosphere_path = os.path.join(input_dir, atmosphere_file)
atmosphere = xr.load_dataset(atmosphere_path, chunks={"expt": 3})
atmosphere

<xarray.Dataset> Size: 2MB
Dimensions:                  (site: 100, expt: 18, layer: 60, level: 61)
Coordinates:
    lon                      (site) float32 400B 27.0 24.0 162.0 ... 250.5 246.0
    lat                      (site) float32 400B -28.5 28.5 31.5 ... -3.0 -24.0
    time                     (site) datetime64[ns] 800B 2014-01-05T06:00:00 ....
    expt_label               (expt) <U49 4kB 'Present day (PD)' ... 'LGM'
Dimensions without coordinates: site, expt, layer, level
Data variables: (12/62)
    sst                      (site) float32 400B nan nan 291.8 ... 298.1 297.0
    pres_layer               (site, layer) float32 24kB 10.0 29.21 ... 1.014e+05
    pres_level               (site, level) float32 24kB 0.01 20.0 ... 1.015e+05
    surface_emissivity       (site) float32 400B 0.98 0.98 0.98 ... 0.98 0.98
    surface_albedo           (site) float32 400B 0.1742 0.391 0.06 ... 0.06 0.06
    solar_zenith_angle       (site) float32 400B 57.45 54.16 ... 70.53 59.51
    ...                       ...
    hfc134a_GM               (expt) float32 72B 80.52 0.0 80.52 ... 421.4 0.0
    c6f14_GM                 (expt) float32 72B 0.279 0.0 0.279 ... 0.5222 0.0
    hcfc141b_GM              (expt) float32 72B 23.81 0.0 23.81 ... 1.285 0.0
    chcl3_GM                 (expt) float32 72B 9.902 6.0 9.902 ... 5.876 0.0
    c2f6_GM                  (expt) float32 72B 4.399 1.289e-06 ... 7.174 0.0
    cfc11eq_GM               (expt) float32 72B 809.2 32.11 ... 1.939e+03 0.0
Attributes: (12/25)
    title:               Atmospheric conditions for off-line radiative transf...
    institution_id:      UColorado
    institution:         University of Colorado, Boulder, CO 80309, USA
    activity_id:         input4MIPs
    Conventions:         CF-1.6
    creation_date:       2019-03-20 16:07:21-0400
    ...                  ...
    nominal_resolution:  10 km
    target_mip:          RFMIP
    variable_id:         multiple
    grid_label:          none
    tracking_id:         hdl:21.14100/f379c294-f7bb-442d-bca7-64661d60780e
    license:             Atmospheric condition data for RFMIP produced by the...

## Computing Gas Optics and Solving Radiation Transfer Equations

Compute the shortwave gas optics using a two-stream approximation and then solve the radiation transfer equations to obtain the fluxes.

In [11]:
gas_optics_sw.gas_optics.compute(atmosphere, problem_type="two-stream")

solver = RTESolver()
fluxes = solver.solve(atmosphere, add_to_input=False)
fluxes

<xarray.Dataset> Size: 3MB
Dimensions:       (site: 100, expt: 18, level: 61)
Coordinates:
    lon           (site) float32 400B 27.0 24.0 162.0 ... 184.5 250.5 246.0
    lat           (site) float32 400B -28.5 28.5 31.5 87.0 ... 58.5 -3.0 -24.0
    time          (site) datetime64[ns] 800B 2014-01-05T06:00:00 ... 2014-12-30
    expt_label    (expt) <U49 4kB 'Present day (PD)' ... 'LGM'
Dimensions without coordinates: site, expt, level
Data variables:
    sw_flux_up    (expt, site, level) float64 878kB 129.1 129.1 ... 30.93 30.85
    sw_flux_down  (expt, site, level) float64 878kB 757.4 757.2 ... 514.7 514.1
    sw_flux_dir   (expt, site, level) float64 878kB 757.4 757.1 ... 467.1 466.5

## Loading Reference Data and Validating the Results

Load the reference datasets for upward and downward shortwave fluxes and validate that the computed fluxes match the reference data within the specified tolerance.

In [10]:
rsu_reference = f"{ref_dir}/rsu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rsd_reference = f"{ref_dir}/rsd_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rsu = xr.load_dataset(rsu_reference, decode_cf=False)
rsd = xr.load_dataset(rsd_reference, decode_cf=False)

assert np.isclose(fluxes["sw_flux_up"], rsu["rsu"], atol=ERROR_TOLERANCE).all(), "Shortwave flux up mismatch"
assert np.isclose(fluxes["sw_flux_down"], rsd["rsd"], atol=ERROR_TOLERANCE).all(), "Shortwave flux down mismatch"

print("Shortwave radiation transfer calculations validated successfully!")

Shortwave radiation transfer calculations validated successfully!
